In [7]:
# Usando a versão 3.10.11 do python

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from pathlib import Path
from datetime import timedelta
import os
pd.options.mode.chained_assignment = None

# Caminho da base
base_diretório = Path(os.getcwd())
diretório_princapl = base_diretório.parent
caminho = diretório_princapl / 'Base' / 'Base Ibovespa Tratada.csv'

print(f'diretorio: {diretório_princapl}')
print(f'caminho: {caminho}')

diretorio: c:\Users\YUPOPIC\OneDrive - Daimler Truck\000 - Pessoal\070 - VSCODE\072 - Tech Challenge\Fase 2\previsao-ibovespa
caminho: c:\Users\YUPOPIC\OneDrive - Daimler Truck\000 - Pessoal\070 - VSCODE\072 - Tech Challenge\Fase 2\previsao-ibovespa\Base\Base Ibovespa Tratada.csv


In [ ]:
acertos = 0
idx = 1

while idx <= 30:

    # Carrega a base
    acao = pd.read_csv(caminho)
    acao['Data'] = pd.to_datetime(acao['Data'])

    # Ordena por data
    acao = acao.sort_values(by="Data")

    # Define data de ontem e anteontem
    ultimo_dia = acao['Data'].max()
    penultimo_dia = acao['Data'].iloc[-2]

    # print(f'Ontem: {ultimo_dia}\nAnteontem: {penultimo_dia}')

    # Filtrando base que vai ser usada para treino até o penultimo dia da base
    acao_treino = acao#[acao['Data'] < ultimo_dia]
    cotacao_treino = acao_treino['Fechamento'].to_numpy().reshape(-1, 1)

    # Armazenar tamanho dos dados de treinamento
    tamanho_dados_treinamento = int(len(cotacao_treino) * 1)

    # Escala os dados
    escalador = MinMaxScaler(feature_range=(0, 1))
    dados_escalados = escalador.fit_transform(cotacao_treino)

    # Cria dados de entrada para o modelo
    treinamento_x = []
    treinamento_y = []

    for i in range(60, len(dados_escalados)):
        treinamento_x.append(dados_escalados[i-60:i, 0])
        treinamento_y.append(dados_escalados[i, 0])

    treinamento_x, treinamento_y = np.array(treinamento_x), np.array(treinamento_y)
    treinamento_x = treinamento_x.reshape(treinamento_x.shape[0], treinamento_x.shape[1], 1)

    # Define e treina o modelo
    modelo = Sequential()
    modelo.add(LSTM(100, return_sequences=True, input_shape=(treinamento_x.shape[1], 1)))
    modelo.add(LSTM(50, return_sequences=False))
    modelo.add(Dense(25))
    modelo.add(Dense(1))
    modelo.compile(optimizer="adam", loss="mean_squared_error")
    modelo.fit(treinamento_x, treinamento_y, batch_size=10, epochs=20)

    # Pega os últimos 60 dias até ANTEONTEM para previsão
    ultimos_60 = dados_escalados[-60:]
    entrada_predicao = ultimos_60.reshape(1, 60, 1)

    predicao = modelo.predict(entrada_predicao)
    predicao = escalador.inverse_transform(predicao)

    # print(f'Valor previsto: {predicao[0][0]:.2f}')

    print(f'\nTeste {idx}')
    # ---- Descomentar as quatro linhas de baixo caso for fazer teste do valor previsto do ultimo dia da base ----
    valor_real = acao[acao['Data'] == ultimo_dia]['Fechamento'].values[0]
    valor_anterior = acao[acao['Data'] == penultimo_dia]['Fechamento'].values[0]
    variacao_real = (valor_real - valor_anterior) / valor_anterior * 100
    variacao_predita = (predicao[0][0] - valor_anterior) / valor_anterior * 100
    # -----------------------------------------------------------------------------

    # ---- Descomentar as quatro linhas de baixo caso for prever do dia seguinte ----
    #valor_real = 137002	# Valor de fechamento do ibovespa no dia 04/06/2025 (dia seguinte do ultimo que tem na base)
    #valor_anterior = acao[acao['Data'] == ultimo_dia]['Fechamento'].values[0]
    #variacao_real = (valor_real - valor_anterior) / valor_anterior * 100
    #variacao_predita = (predicao[0][0] - valor_anterior) / valor_anterior * 100
    # -----------------------------------------------------------------------------

    # Direção correta?
    direcao_correta = (variacao_real * variacao_predita) > 0  # ambos com mesmo sinal

    # Criar df final
    df_previsao = pd.DataFrame([{
        'Data': ultimo_dia.date(), # Data do penultimo dia da base
        #'Data Previsão': ultimo_dia.date() + timedelta(days=1), # Data do dia seguinte do ultimo dia da base
        'Valor Anterior': f"{valor_anterior:.2f}",
        'Valor Real': f"{valor_real:.2f}",
        'Valor Previsto': f"{predicao[0][0]:.2f}",
        'Variação Real (%)': f"{variacao_real:.3f}",
        'Variação Prevista (%)': f"{variacao_predita:.3f}",
        'Direção Correta?': 'Sim' if direcao_correta else 'Não'
    }])

    # Ajustar colunas numéricas
    colunas_para_converter = ['Valor Real', 'Valor Previsto', 'Variação Real (%)', 'Variação Prevista (%)']
    for coluna in colunas_para_converter:
        df_previsao[coluna] = df_previsao[coluna].astype(float)
    
    df_previsao
    
    if df_previsao['Direção Correta?'].iloc[0] == 'Sim':
        acertos += 1
        print(f'total de acertos: {acertos}\n')

    idx +=1